Factors: A.1.7, A.1.8, A.10, A.2.18(5-year sales growth rank), A.2.19 (Sales Growth)

In [1]:
import pandas as pd
import numpy as np
from pandasql import sqldf

In [2]:
data = pd.read_csv("merged_final.csv")
#data

In [3]:
final = pd.read_csv("merged_final.csv")

In [4]:
data = data[['permno', 'yyyymm', 'monthid', 'ticker', 'cusip', 'RET']]
data

,permno,yyyymm,monthid,ticker,cusip,RET
0,10026,198603.0,75.0,JJSF,466032109,0.133333
1,10026,198604.0,76.0,JJSF,466032109,0.070588
2,10026,198605.0,77.0,JJSF,466032109,0.406593
3,10026,198606.0,78.0,JJSF,466032109,-0.156250
4,10026,198607.0,79.0,JJSF,466032109,-0.375000
...,...,...,...,...,...,...
401583,93429,201802.0,458.0,CBOE,12503M108,-0.166530
401584,93429,201803.0,459.0,CBOE,12503M108,0.021070
401585,93429,201804.0,460.0,CBOE,12503M108,-0.064154
401586,93429,201805.0,461.0,CBOE,12503M108,-0.083817


# Functions

In [5]:
def quantile(df, inVar, outVar, quantCount):
    groups = df.groupby(['monthid'])[inVar]
    df[outVar] = groups.transform(lambda x: pd.qcut(x, q=quantCount, labels=False, duplicates='drop'))
    return df

# Revenue Surprises A.1.7

In [6]:
compustat_rev = pd.read_csv("compustat a-1-7.csv")
#compustat_rev

In [7]:
revDf = compustat_rev[['datadate', 'tic', 'cusip', 'saleq', 'cshprq', 'rdq', 'ajexq']]
revDf["yyyymm"] = pd.DatetimeIndex(revDf['datadate']).year * 100 + pd.DatetimeIndex(revDf['datadate']).month
revDf["monthid"] = (pd.DatetimeIndex(revDf['datadate']).year-1980)*12 + pd.DatetimeIndex(revDf['datadate']).month
revDf['rdqmonthid'] = (pd.DatetimeIndex(revDf['rdq']).year-1980)*12 + pd.DatetimeIndex(revDf['rdq']).month
revDf

C:\Users\schen\AppData\Local\Temp\ipykernel_22892\1098136396.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  revDf["yyyymm"] = pd.DatetimeIndex(revDf['datadate']).year * 100 + pd.DatetimeIndex(revDf['datadate']).month
C:\Users\schen\AppData\Local\Temp\ipykernel_22892\1098136396.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  revDf["monthid"] = (pd.DatetimeIndex(revDf['datadate']).year-1980)*12 + pd.DatetimeIndex(revDf['datadate']).month
C:\Users\schen\AppData\Local\Temp\ipykernel_22892\1098136396.py

,datadate,tic,cusip,saleq,cshprq,rdq,ajexq,yyyymm,monthid,rdqmonthid
0,1984-02-29,AIR,000361105,44.706,4.510,1984-03-28,3.375,198402,50,51.0
1,1984-05-31,AIR,000361105,51.374,6.002,1984-07-26,3.375,198405,53,55.0
2,1984-08-31,AIR,000361105,52.723,6.013,1984-09-26,3.375,198408,56,57.0
3,1984-11-30,AIR,000361105,55.156,6.023,1984-12-17,3.375,198411,59,60.0
4,1985-02-28,AIR,000361105,50.939,6.030,1985-03-26,3.375,198502,62,63.0
...,...,...,...,...,...,...,...,...,...,...
161325,2020-12-31,ACA,039653100,458.900,48.000,2021-02-24,1.000,202012,492,494.0
161326,2021-03-31,ACA,039653100,440.400,48.000,2021-04-29,1.000,202103,495,496.0
161327,2021-06-30,ACA,039653100,507.400,48.100,2021-08-04,1.000,202106,498,500.0
161328,2021-09-30,ACA,039653100,559.100,48.200,2021-11-03,1.000,202109,501,503.0


In [8]:
revDf = sqldf("SELECT a.*, b.saleq, b.cshprq, b.ajexq, b.rdq, b.rdqmonthid \
            FROM data as a \
            LEFT JOIN revDf as b \
            ON a.ticker = b.tic and a.monthid > b.rdqmonthid and a.monthid <= b.rdqmonthid + 3")
revDf = revDf.drop_duplicates(subset=['permno', 'yyyymm'], keep='last')
revDf

,permno,yyyymm,monthid,ticker,cusip,RET,saleq,cshprq,ajexq,rdq,rdqmonthid
0,10026,198603.0,75.0,JJSF,466032109,0.133333,NaN,NaN,NaN,None,NaN
1,10026,198604.0,76.0,JJSF,466032109,0.070588,NaN,NaN,NaN,None,NaN
2,10026,198605.0,77.0,JJSF,466032109,0.406593,NaN,NaN,NaN,None,NaN
3,10026,198606.0,78.0,JJSF,466032109,-0.156250,NaN,NaN,NaN,None,NaN
4,10026,198607.0,79.0,JJSF,466032109,-0.375000,NaN,NaN,NaN,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...
419448,93429,201802.0,458.0,CBOE,12503M108,-0.166530,611.4,112.3,1.0,2017-11-07,455.0
419449,93429,201803.0,459.0,CBOE,12503M108,0.021070,620.7,112.3,1.0,2018-02-09,458.0
419450,93429,201804.0,460.0,CBOE,12503M108,-0.064154,620.7,112.3,1.0,2018-02-09,458.0
419451,93429,201805.0,461.0,CBOE,12503M108,-0.083817,620.7,112.3,1.0,2018-02-09,458.0


In [9]:
revDf['rev/share'] = revDf['saleq'] / (revDf['cshprq']*revDf['ajexq'])
revDf['revChange'] = revDf['rev/share'].diff(12)
revDf['std'] = revDf['revChange'].rolling(window=24, min_periods=18).std()
revDf['Rs'] = revDf['revChange'] / revDf['std']
revDf

,permno,yyyymm,monthid,ticker,cusip,RET,saleq,cshprq,ajexq,rdq,rdqmonthid,rev/share,revChange,std,Rs
0,10026,198603.0,75.0,JJSF,466032109,0.133333,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
1,10026,198604.0,76.0,JJSF,466032109,0.070588,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
2,10026,198605.0,77.0,JJSF,466032109,0.406593,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
3,10026,198606.0,78.0,JJSF,466032109,-0.156250,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
4,10026,198607.0,79.0,JJSF,466032109,-0.375000,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419448,93429,201802.0,458.0,CBOE,12503M108,-0.166530,611.4,112.3,1.0,2017-11-07,455.0,5.444346,NaN,1.438787,NaN
419449,93429,201803.0,459.0,CBOE,12503M108,0.021070,620.7,112.3,1.0,2018-02-09,458.0,5.527159,3.179863,1.473759,2.157655
419450,93429,201804.0,460.0,CBOE,12503M108,-0.064154,620.7,112.3,1.0,2018-02-09,458.0,5.527159,3.179863,1.492581,2.130446
419451,93429,201805.0,461.0,CBOE,12503M108,-0.083817,620.7,112.3,1.0,2018-02-09,458.0,5.527159,3.179863,1.505353,2.112371


In [10]:
#revDf = revDf[revDf['Rs'] != 0]
revDf = revDf.dropna(subset=['Rs'])
revDf

,permno,yyyymm,monthid,ticker,cusip,RET,saleq,cshprq,ajexq,rdq,rdqmonthid,rev/share,revChange,std,Rs
48,10026,199001.0,121.0,JJSF,466032109,-0.151515,26.946,8.576,2.0,1989-11-16,119.0,1.571012,-0.057270,0.255045,-0.224550
49,10026,199002.0,122.0,JJSF,466032109,-0.166667,26.946,8.576,2.0,1989-11-16,119.0,1.571012,-0.057270,0.254455,-0.225071
50,10026,199003.0,123.0,JJSF,466032109,-0.014286,20.301,8.541,2.0,1990-02-05,122.0,1.188444,-0.332813,0.272965,-1.219253
51,10026,199004.0,124.0,JJSF,466032109,0.057971,20.301,8.541,2.0,1990-02-05,122.0,1.188444,-0.332813,0.286539,-1.161494
52,10026,199005.0,125.0,JJSF,466032109,-0.054795,20.301,8.541,2.0,1990-02-05,122.0,1.188444,-0.332813,0.289748,-1.148628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419447,93429,201801.0,457.0,CBOE,12503M108,0.078658,611.400,112.300,1.0,2017-11-07,455.0,5.444346,3.368932,1.438787,2.341509
419449,93429,201803.0,459.0,CBOE,12503M108,0.021070,620.700,112.300,1.0,2018-02-09,458.0,5.527159,3.179863,1.473759,2.157655
419450,93429,201804.0,460.0,CBOE,12503M108,-0.064154,620.700,112.300,1.0,2018-02-09,458.0,5.527159,3.179863,1.492581,2.130446
419451,93429,201805.0,461.0,CBOE,12503M108,-0.083817,620.700,112.300,1.0,2018-02-09,458.0,5.527159,3.179863,1.505353,2.112371


In [11]:
revDf = quantile(revDf, 'Rs', 'Decile', 10)
revDf

C:\Users\schen\AppData\Local\Temp\ipykernel_22892\3623145156.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[outVar] = groups.transform(lambda x: pd.qcut(x, q=quantCount, labels=False, duplicates='drop'))


,permno,yyyymm,monthid,ticker,cusip,RET,saleq,cshprq,ajexq,rdq,rdqmonthid,rev/share,revChange,std,Rs,Decile
48,10026,199001.0,121.0,JJSF,466032109,-0.151515,26.946,8.576,2.0,1989-11-16,119.0,1.571012,-0.057270,0.255045,-0.224550,1.0
49,10026,199002.0,122.0,JJSF,466032109,-0.166667,26.946,8.576,2.0,1989-11-16,119.0,1.571012,-0.057270,0.254455,-0.225071,1.0
50,10026,199003.0,123.0,JJSF,466032109,-0.014286,20.301,8.541,2.0,1990-02-05,122.0,1.188444,-0.332813,0.272965,-1.219253,0.0
51,10026,199004.0,124.0,JJSF,466032109,0.057971,20.301,8.541,2.0,1990-02-05,122.0,1.188444,-0.332813,0.286539,-1.161494,0.0
52,10026,199005.0,125.0,JJSF,466032109,-0.054795,20.301,8.541,2.0,1990-02-05,122.0,1.188444,-0.332813,0.289748,-1.148628,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419447,93429,201801.0,457.0,CBOE,12503M108,0.078658,611.400,112.300,1.0,2017-11-07,455.0,5.444346,3.368932,1.438787,2.341509,7.0
419449,93429,201803.0,459.0,CBOE,12503M108,0.021070,620.700,112.300,1.0,2018-02-09,458.0,5.527159,3.179863,1.473759,2.157655,6.0
419450,93429,201804.0,460.0,CBOE,12503M108,-0.064154,620.700,112.300,1.0,2018-02-09,458.0,5.527159,3.179863,1.492581,2.130446,6.0
419451,93429,201805.0,461.0,CBOE,12503M108,-0.083817,620.700,112.300,1.0,2018-02-09,458.0,5.527159,3.179863,1.505353,2.112371,6.0


In [12]:
revDf

,permno,yyyymm,monthid,ticker,cusip,RET,saleq,cshprq,ajexq,rdq,rdqmonthid,rev/share,revChange,std,Rs,Decile
48,10026,199001.0,121.0,JJSF,466032109,-0.151515,26.946,8.576,2.0,1989-11-16,119.0,1.571012,-0.057270,0.255045,-0.224550,1.0
49,10026,199002.0,122.0,JJSF,466032109,-0.166667,26.946,8.576,2.0,1989-11-16,119.0,1.571012,-0.057270,0.254455,-0.225071,1.0
50,10026,199003.0,123.0,JJSF,466032109,-0.014286,20.301,8.541,2.0,1990-02-05,122.0,1.188444,-0.332813,0.272965,-1.219253,0.0
51,10026,199004.0,124.0,JJSF,466032109,0.057971,20.301,8.541,2.0,1990-02-05,122.0,1.188444,-0.332813,0.286539,-1.161494,0.0
52,10026,199005.0,125.0,JJSF,466032109,-0.054795,20.301,8.541,2.0,1990-02-05,122.0,1.188444,-0.332813,0.289748,-1.148628,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
419447,93429,201801.0,457.0,CBOE,12503M108,0.078658,611.400,112.300,1.0,2017-11-07,455.0,5.444346,3.368932,1.438787,2.341509,7.0
419449,93429,201803.0,459.0,CBOE,12503M108,0.021070,620.700,112.300,1.0,2018-02-09,458.0,5.527159,3.179863,1.473759,2.157655,6.0
419450,93429,201804.0,460.0,CBOE,12503M108,-0.064154,620.700,112.300,1.0,2018-02-09,458.0,5.527159,3.179863,1.492581,2.130446,6.0
419451,93429,201805.0,461.0,CBOE,12503M108,-0.083817,620.700,112.300,1.0,2018-02-09,458.0,5.527159,3.179863,1.505353,2.112371,6.0


In [13]:
final = pd.merge(final, revDf[['permno', 'yyyymm', 'Rs']], on=['permno', 'yyyymm'], how='inner')
final

,permno,yyyymm,monthid,ticker,gvkey,cusip,naics,gsubind,IM,range_20,...,ret_f4,ret_f5,ret_f6,ret_f7,ret_f8,ret_f9,ret_f10,ret_f11,ret_f12,Rs
0,10026,199001.0,121.0,JJSF,12825,466032109,311812,30202030,0.511680,0.407641,...,-0.054795,-0.043478,0.030303,-0.088235,-0.209677,-0.040816,0.489362,-0.100000,0.015873,-0.224550
1,10026,199002.0,122.0,JJSF,12825,466032109,311812,30202030,0.524748,0.444449,...,-0.043478,0.030303,-0.088235,-0.209677,-0.040816,0.489362,-0.100000,0.015873,0.375000,-0.225071
2,10026,199003.0,123.0,JJSF,12825,466032109,311812,30202030,0.299829,0.520924,...,0.030303,-0.088235,-0.209677,-0.040816,0.489362,-0.100000,0.015873,0.375000,0.056818,-1.219253
3,10026,199004.0,124.0,JJSF,12825,466032109,311812,30202030,0.111335,0.303839,...,-0.088235,-0.209677,-0.040816,0.489362,-0.100000,0.015873,0.375000,0.056818,0.193548,-1.161494
4,10026,199005.0,125.0,JJSF,12825,466032109,311812,30202030,0.184506,0.378662,...,-0.209677,-0.040816,0.489362,-0.100000,0.015873,0.375000,0.056818,0.193548,-0.045045,-1.148628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298070,93429,201801.0,457.0,CBOE,184500,12503M108,523210,40203040,0.568230,0.089902,...,-0.083817,0.066728,-0.066686,0.040976,-0.048016,0.176011,-0.043598,-0.090968,-0.046612,2.341509
298071,93429,201803.0,459.0,CBOE,184500,12503M108,523210,40203040,0.562365,0.340422,...,-0.066686,0.040976,-0.048016,0.176011,-0.043598,-0.090968,-0.046612,0.031629,-0.004900,2.157655
298072,93429,201804.0,460.0,CBOE,184500,12503M108,523210,40203040,0.544807,0.193830,...,0.040976,-0.048016,0.176011,-0.043598,-0.090968,-0.046612,0.031629,-0.004900,0.064648,2.130446
298073,93429,201805.0,461.0,CBOE,184500,12503M108,523210,40203040,0.455098,0.122062,...,-0.048016,0.176011,-0.043598,-0.090968,-0.046612,0.031629,-0.004900,0.064648,0.071253,2.112371


# Tax Expense Surprises A.1.8

In [14]:
compustat_tax = pd.read_csv("compustat a-1-8.csv")
#compustat_tax

In [15]:
taxDf = compustat_tax[['datadate', 'tic', 'cusip', 'atq', 'cshprq', 'txtq', 'ajexq']]

In [16]:
taxDf["yyyymm"] = pd.DatetimeIndex(taxDf['datadate']).year * 100 + pd.DatetimeIndex(taxDf['datadate']).month
taxDf["monthid"] = (pd.DatetimeIndex(taxDf['datadate']).year-1980)*12 + pd.DatetimeIndex(taxDf['datadate']).month

C:\Users\schen\AppData\Local\Temp\ipykernel_22892\999723739.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taxDf["yyyymm"] = pd.DatetimeIndex(taxDf['datadate']).year * 100 + pd.DatetimeIndex(taxDf['datadate']).month
C:\Users\schen\AppData\Local\Temp\ipykernel_22892\999723739.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  taxDf["monthid"] = (pd.DatetimeIndex(taxDf['datadate']).year-1980)*12 + pd.DatetimeIndex(taxDf['datadate']).month


In [17]:
taxDf = sqldf("SELECT a.*, b.atq, b.cshprq, b.txtq, b.ajexq \
            FROM data as a \
            LEFT JOIN taxDf as b \
            ON a.ticker = b.tic and a.monthid >= b.monthid + 4 and a.monthid <= b.monthid + 6")

In [18]:
taxDf['taxExp'] = taxDf['txtq'] / (taxDf['cshprq'] * taxDf['ajexq'])
taxDf['assets/share'] = taxDf['atq'] / (taxDf['cshprq'] * taxDf['ajexq'])
taxDf['tes'] = (taxDf['taxExp'] - taxDf['taxExp'].shift(12)) * taxDf['assets/share'].shift(12)
#taxDf = taxDf[taxDf['tes'] != 0]
taxDf = taxDf.dropna(subset=['tes'])

In [19]:
taxDf = quantile(taxDf, 'tes', 'Decile', 10)
taxDf

C:\Users\schen\anaconda3\lib\site-packages\numpy\lib\function_base.py:4486: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)
C:\Users\schen\anaconda3\lib\site-packages\numpy\lib\function_base.py:4488: RuntimeWarning: invalid value encountered in multiply
  lerp_interpolation = asanyarray(add(a, diff_b_a * t, out=out))


,permno,yyyymm,monthid,ticker,cusip,RET,atq,cshprq,txtq,ajexq,taxExp,assets/share,tes,Decile
12,10026,198703.0,87.0,JJSF,466032109,0.030769,23.286,2.918,0.311,4.0,0.026645,1.995031,-0.022394,2.0
16,10026,198707.0,91.0,JJSF,466032109,0.114504,25.584,2.985,0.402,4.0,0.033668,2.142714,0.027242,5.0
17,10026,198708.0,92.0,JJSF,466032109,0.123288,25.584,2.985,0.402,4.0,0.033668,2.142714,0.027242,6.0
18,10026,198709.0,93.0,JJSF,466032109,0.060976,25.584,2.985,0.402,4.0,0.033668,2.142714,0.027242,6.0
19,10026,198710.0,94.0,JJSF,466032109,-0.379310,46.717,2.985,0.415,4.0,0.034757,3.912647,0.033756,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401802,93429,201802.0,458.0,CBOE,12503M108,-0.166530,5228.100,112.300,45.600,1.0,0.406055,46.554764,0.407873,5.0
401803,93429,201803.0,459.0,CBOE,12503M108,0.021070,5228.100,112.300,45.600,1.0,0.406055,46.554764,0.407873,5.0
401804,93429,201804.0,460.0,CBOE,12503M108,-0.064154,5265.700,112.300,-153.000,1.0,-1.362422,46.889581,-10.138196,2.0
401805,93429,201805.0,461.0,CBOE,12503M108,-0.083817,5265.700,112.300,-153.000,1.0,-1.362422,46.889581,-10.138196,3.0


In [20]:
taxDf.tail(50)

,permno,yyyymm,monthid,ticker,cusip,RET,atq,cshprq,txtq,ajexq,taxExp,assets/share,tes,Decile
401757,93429,201405.0,413.0,CBOE,12503M108,-0.046852,441.589,87.061,26.043,1.0,0.299135,5.072179,0.286169,5.0
401758,93429,201406.0,414.0,CBOE,12503M108,-0.029006,441.589,87.061,26.043,1.0,0.299135,5.072179,0.286169,5.0
401759,93429,201407.0,415.0,CBOE,12503M108,-0.015038,412.844,86.453,32.519,1.0,0.376147,4.775358,0.349102,4.0
401760,93429,201408.0,416.0,CBOE,12503M108,0.098308,412.844,86.453,32.519,1.0,0.376147,4.775358,0.349102,4.0
401761,93429,201409.0,417.0,CBOE,12503M108,0.009430,412.844,86.453,32.519,1.0,0.376147,4.775358,0.349102,4.0
401762,93429,201410.0,418.0,CBOE,12503M108,0.101168,385.781,85.831,26.414,1.0,0.307744,4.494658,-0.103983,3.0
401763,93429,201411.0,419.0,CBOE,12503M108,0.020020,385.781,85.831,26.414,1.0,0.307744,4.494658,-0.103983,3.0
401764,93429,201412.0,420.0,CBOE,12503M108,0.058588,385.781,85.831,26.414,1.0,0.307744,4.494658,-0.103983,3.0
401765,93429,201501.0,421.0,CBOE,12503M108,0.016556,367.086,85.051,26.446,1.0,0.310943,4.316069,0.040153,4.0
401766,93429,201502.0,422.0,CBOE,12503M108,-0.065612,367.086,85.051,26.446,1.0,0.310943,4.316069,0.040153,3.0


In [21]:
final = pd.merge(final, taxDf[['permno', 'yyyymm', 'tes']], on=['permno', 'yyyymm'], how='inner')
final

,permno,yyyymm,monthid,ticker,gvkey,cusip,naics,gsubind,IM,range_20,...,ret_f5,ret_f6,ret_f7,ret_f8,ret_f9,ret_f10,ret_f11,ret_f12,Rs,tes
0,10026,199001.0,121.0,JJSF,12825,466032109,311812,30202030,0.511680,0.407641,...,-0.043478,0.030303,-0.088235,-0.209677,-0.040816,0.489362,-0.100000,0.015873,-0.224550,0.068135
1,10026,199002.0,122.0,JJSF,12825,466032109,311812,30202030,0.524748,0.444449,...,0.030303,-0.088235,-0.209677,-0.040816,0.489362,-0.100000,0.015873,0.375000,-0.225071,0.068135
2,10026,199003.0,123.0,JJSF,12825,466032109,311812,30202030,0.299829,0.520924,...,-0.088235,-0.209677,-0.040816,0.489362,-0.100000,0.015873,0.375000,0.056818,-1.219253,0.068135
3,10026,199004.0,124.0,JJSF,12825,466032109,311812,30202030,0.111335,0.303839,...,-0.209677,-0.040816,0.489362,-0.100000,0.015873,0.375000,0.056818,0.193548,-1.161494,-0.125056
4,10026,199005.0,125.0,JJSF,12825,466032109,311812,30202030,0.184506,0.378662,...,-0.040816,0.489362,-0.100000,0.015873,0.375000,0.056818,0.193548,-0.045045,-1.148628,-0.125056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297409,93429,201801.0,457.0,CBOE,184500,12503M108,523210,40203040,0.568230,0.089902,...,0.066728,-0.066686,0.040976,-0.048016,0.176011,-0.043598,-0.090968,-0.046612,2.341509,0.407873
297410,93429,201803.0,459.0,CBOE,184500,12503M108,523210,40203040,0.562365,0.340422,...,0.040976,-0.048016,0.176011,-0.043598,-0.090968,-0.046612,0.031629,-0.004900,2.157655,0.407873
297411,93429,201804.0,460.0,CBOE,184500,12503M108,523210,40203040,0.544807,0.193830,...,-0.048016,0.176011,-0.043598,-0.090968,-0.046612,0.031629,-0.004900,0.064648,2.130446,-10.138196
297412,93429,201805.0,461.0,CBOE,184500,12503M108,523210,40203040,0.455098,0.122062,...,0.176011,-0.043598,-0.090968,-0.046612,0.031629,-0.004900,0.064648,0.071253,2.112371,-10.138196


# A.1.10

In [22]:
compustat_earnings = pd.read_csv("compustat a-1-10.csv")
#compustat_earnings

In [23]:
earnDf = compustat_earnings[['datadate', 'tic', 'cusip', 'rdq', 'ibq']]
earnDf["yyyymm"] = pd.DatetimeIndex(earnDf['datadate']).year * 100 + pd.DatetimeIndex(earnDf['datadate']).month
earnDf["monthid"] = (pd.DatetimeIndex(earnDf['datadate']).year-1980)*12 + pd.DatetimeIndex(earnDf['datadate']).month
earnDf['rdqmonthid'] = (pd.DatetimeIndex(earnDf['rdq']).year-1980)*12 + pd.DatetimeIndex(earnDf['rdq']).month

C:\Users\schen\AppData\Local\Temp\ipykernel_22892\428864831.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  earnDf["yyyymm"] = pd.DatetimeIndex(earnDf['datadate']).year * 100 + pd.DatetimeIndex(earnDf['datadate']).month
C:\Users\schen\AppData\Local\Temp\ipykernel_22892\428864831.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  earnDf["monthid"] = (pd.DatetimeIndex(earnDf['datadate']).year-1980)*12 + pd.DatetimeIndex(earnDf['datadate']).month
C:\Users\schen\AppData\Local\Temp\ipykernel_22892\428864831

In [24]:
earnDf = sqldf("SELECT a.*, b.ibq, b.rdq, b.rdqmonthid \
            FROM data as a \
            LEFT JOIN earnDf as b \
            ON a.ticker = b.tic and a.monthid > b.rdqmonthid and a.monthid <= b.rdqmonthid + 3")
earnDf = earnDf.drop_duplicates(subset=['permno', 'yyyymm'], keep='last')
earnDf.tail(50)

,permno,yyyymm,monthid,ticker,cusip,RET,ibq,rdq,rdqmonthid
419400,93429,201405.0,413.0,CBOE,12503M108,-0.046852,46.110,2014-02-07,410.0
419401,93429,201406.0,414.0,CBOE,12503M108,-0.029006,49.024,2014-05-06,413.0
419402,93429,201407.0,415.0,CBOE,12503M108,-0.015038,49.024,2014-05-06,413.0
419403,93429,201408.0,416.0,CBOE,12503M108,0.098308,49.024,2014-05-06,413.0
419404,93429,201409.0,417.0,CBOE,12503M108,0.009430,42.981,2014-08-01,416.0
419405,93429,201410.0,418.0,CBOE,12503M108,0.101168,42.981,2014-08-01,416.0
419407,93429,201411.0,419.0,CBOE,12503M108,0.020020,48.367,2014-10-31,418.0
419408,93429,201412.0,420.0,CBOE,12503M108,0.058588,48.367,2014-10-31,418.0
419409,93429,201501.0,421.0,CBOE,12503M108,0.016556,48.367,2014-10-31,418.0
419410,93429,201502.0,422.0,CBOE,12503M108,-0.065612,NaN,None,NaN


In [25]:
earnDf = earnDf.dropna(subset=['ibq'])
earnDf.reset_index(drop=True, inplace=True)
#earnDf

In [26]:
earnDf.tail(50)

,permno,yyyymm,monthid,ticker,cusip,RET,ibq,rdq,rdqmonthid
330592,93429,201402.0,410.0,CBOE,12503M108,0.042099,41.356,2013-11-01,407.0
330593,93429,201403.0,411.0,CBOE,12503M108,0.047566,46.110,2014-02-07,410.0
330594,93429,201404.0,412.0,CBOE,12503M108,-0.057244,46.110,2014-02-07,410.0
330595,93429,201405.0,413.0,CBOE,12503M108,-0.046852,46.110,2014-02-07,410.0
330596,93429,201406.0,414.0,CBOE,12503M108,-0.029006,49.024,2014-05-06,413.0
330597,93429,201407.0,415.0,CBOE,12503M108,-0.015038,49.024,2014-05-06,413.0
330598,93429,201408.0,416.0,CBOE,12503M108,0.098308,49.024,2014-05-06,413.0
330599,93429,201409.0,417.0,CBOE,12503M108,0.009430,42.981,2014-08-01,416.0
330600,93429,201410.0,418.0,CBOE,12503M108,0.101168,42.981,2014-08-01,416.0
330601,93429,201411.0,419.0,CBOE,12503M108,0.020020,48.367,2014-10-31,418.0


In [27]:
earnDf['Nei'] = 0
#earnDf


In [28]:
for i in range(len(earnDf)):
    if i - 12 >= 0 and earnDf.loc[i-12, 'ticker'] == earnDf.loc[i, 'ticker'] and earnDf.loc[i-12, 'ibq'] < earnDf.loc[i, 'ibq']:
        earnDf.loc[i, 'Nei'] = min(earnDf.loc[i-12, 'Nei'] + 1, 8)
earnDf

,permno,yyyymm,monthid,ticker,cusip,RET,ibq,rdq,rdqmonthid,Nei
0,10026,198705.0,89.0,JJSF,466032109,-0.042373,0.533,1987-04-28,88.0,0
1,10026,198706.0,90.0,JJSF,466032109,0.159292,0.533,1987-04-28,88.0,0
2,10026,198707.0,91.0,JJSF,466032109,0.114504,0.533,1987-04-28,88.0,0
3,10026,198709.0,93.0,JJSF,466032109,0.060976,0.803,1987-08-05,92.0,0
4,10026,198710.0,94.0,JJSF,466032109,-0.379310,0.803,1987-08-05,92.0,0
...,...,...,...,...,...,...,...,...,...,...
330637,93429,201802.0,458.0,CBOE,12503M108,-0.166530,60.300,2017-11-07,455.0,1
330638,93429,201803.0,459.0,CBOE,12503M108,0.021070,257.100,2018-02-09,458.0,1
330639,93429,201804.0,460.0,CBOE,12503M108,-0.064154,257.100,2018-02-09,458.0,1
330640,93429,201805.0,461.0,CBOE,12503M108,-0.083817,257.100,2018-02-09,458.0,1


In [29]:
earnDf.tail(50)

,permno,yyyymm,monthid,ticker,cusip,RET,ibq,rdq,rdqmonthid,Nei
330592,93429,201402.0,410.0,CBOE,12503M108,0.042099,41.356,2013-11-01,407.0,0
330593,93429,201403.0,411.0,CBOE,12503M108,0.047566,46.110,2014-02-07,410.0,3
330594,93429,201404.0,412.0,CBOE,12503M108,-0.057244,46.110,2014-02-07,410.0,3
330595,93429,201405.0,413.0,CBOE,12503M108,-0.046852,46.110,2014-02-07,410.0,3
330596,93429,201406.0,414.0,CBOE,12503M108,-0.029006,49.024,2014-05-06,413.0,3
330597,93429,201407.0,415.0,CBOE,12503M108,-0.015038,49.024,2014-05-06,413.0,4
330598,93429,201408.0,416.0,CBOE,12503M108,0.098308,49.024,2014-05-06,413.0,4
330599,93429,201409.0,417.0,CBOE,12503M108,0.009430,42.981,2014-08-01,416.0,0
330600,93429,201410.0,418.0,CBOE,12503M108,0.101168,42.981,2014-08-01,416.0,0
330601,93429,201411.0,419.0,CBOE,12503M108,0.020020,48.367,2014-10-31,418.0,4


In [30]:
final = pd.merge(final, earnDf[['permno', 'yyyymm', 'Nei']], on=['permno', 'yyyymm'], how='inner')
final

,permno,yyyymm,monthid,ticker,gvkey,cusip,naics,gsubind,IM,range_20,...,ret_f6,ret_f7,ret_f8,ret_f9,ret_f10,ret_f11,ret_f12,Rs,tes,Nei
0,10026,199001.0,121.0,JJSF,12825,466032109,311812,30202030,0.511680,0.407641,...,0.030303,-0.088235,-0.209677,-0.040816,0.489362,-0.100000,0.015873,-0.224550,0.068135,2
1,10026,199002.0,122.0,JJSF,12825,466032109,311812,30202030,0.524748,0.444449,...,-0.088235,-0.209677,-0.040816,0.489362,-0.100000,0.015873,0.375000,-0.225071,0.068135,2
2,10026,199003.0,123.0,JJSF,12825,466032109,311812,30202030,0.299829,0.520924,...,-0.209677,-0.040816,0.489362,-0.100000,0.015873,0.375000,0.056818,-1.219253,0.068135,0
3,10026,199004.0,124.0,JJSF,12825,466032109,311812,30202030,0.111335,0.303839,...,-0.040816,0.489362,-0.100000,0.015873,0.375000,0.056818,0.193548,-1.161494,-0.125056,0
4,10026,199005.0,125.0,JJSF,12825,466032109,311812,30202030,0.184506,0.378662,...,0.489362,-0.100000,0.015873,0.375000,0.056818,0.193548,-0.045045,-1.148628,-0.125056,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297389,93429,201801.0,457.0,CBOE,184500,12503M108,523210,40203040,0.568230,0.089902,...,-0.066686,0.040976,-0.048016,0.176011,-0.043598,-0.090968,-0.046612,2.341509,0.407873,1
297390,93429,201803.0,459.0,CBOE,184500,12503M108,523210,40203040,0.562365,0.340422,...,-0.048016,0.176011,-0.043598,-0.090968,-0.046612,0.031629,-0.004900,2.157655,0.407873,1
297391,93429,201804.0,460.0,CBOE,184500,12503M108,523210,40203040,0.544807,0.193830,...,0.176011,-0.043598,-0.090968,-0.046612,0.031629,-0.004900,0.064648,2.130446,-10.138196,1
297392,93429,201805.0,461.0,CBOE,184500,12503M108,523210,40203040,0.455098,0.122062,...,-0.043598,-0.090968,-0.046612,0.031629,-0.004900,0.064648,0.071253,2.112371,-10.138196,1


# A.2.18 & A.2.19

In [31]:
compustat_sales = pd.read_csv("compustat annual sales.csv")

In [32]:
salesDf = compustat_sales[['datadate', 'tic', 'cusip', 'sale']]
salesDf["yyyymm"] = pd.DatetimeIndex(salesDf['datadate']).year * 100 + pd.DatetimeIndex(salesDf['datadate']).month
salesDf["monthid"] = (pd.DatetimeIndex(salesDf['datadate']).year-1980)*12 + pd.DatetimeIndex(salesDf['datadate']).month
#salesDf

C:\Users\schen\AppData\Local\Temp\ipykernel_22892\1244776288.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  salesDf["yyyymm"] = pd.DatetimeIndex(salesDf['datadate']).year * 100 + pd.DatetimeIndex(salesDf['datadate']).month
C:\Users\schen\AppData\Local\Temp\ipykernel_22892\1244776288.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  salesDf["monthid"] = (pd.DatetimeIndex(salesDf['datadate']).year-1980)*12 + pd.DatetimeIndex(salesDf['datadate']).month


In [33]:
salesDf = sqldf("SELECT b.*, a.sale, a.monthid as salemonthid \
                FROM salesDf as a \
                LEFT JOIN data as b \
                ON a.tic = b.ticker and a.monthid + 1 = b.monthid")
#salesDf

In [34]:
salesDf['salesgrowth'] = (salesDf['sale'] / salesDf['sale'].shift(1)) - 1
salesDf.loc[salesDf['ticker'] != salesDf['ticker'].shift(1), 'salesgrowth'] = 0
salesDf = salesDf.dropna(subset=['ticker'])
salesDf = salesDf.loc[salesDf['sale'] >= 0]
#salesDf

In [35]:
salesDf = quantile(salesDf, 'salesgrowth', 'annual decile', 10)
#salesDf

C:\Users\schen\anaconda3\lib\site-packages\numpy\lib\function_base.py:4486: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


In [36]:
# Sr is 5 year annual sales growth rank
salesDf.reset_index(drop=True, inplace=True)
salesDf['Sr'] = 0
for i in range(len(salesDf)):
    if i - 5 >= 0 and salesDf.loc[i-5, 'ticker'] == salesDf.loc[i, 'ticker']:
        numerator = 0
        for j in range(1, 6):
            numerator += (6 - j) * salesDf.loc[i-j, 'annual decile']
        denominator = 15
        salesDf.loc[i, 'Sr'] = numerator/denominator
#salesDf.head(10)

In [37]:
salesDf = quantile(salesDf, 'Sr', '5 year decile', 10)
salesDf.tail(50)

,permno,yyyymm,monthid,ticker,cusip,RET,sale,salemonthid,salesgrowth,annual decile,Sr,5 year decile
28897,90164.0,200901.0,349.0,GHL,395259104,-0.068081,221.873,348,NaN,NaN,0.000000,0.0
28898,90164.0,201001.0,361.0,GHL,395259104,-0.030409,298.645,360,NaN,NaN,NaN,NaN
28899,90164.0,201101.0,373.0,GHL,395259104,-0.150098,278.329,372,NaN,NaN,NaN,NaN
28900,90164.0,201201.0,385.0,GHL,395259104,0.280176,293.993,384,NaN,NaN,NaN,NaN
28901,90164.0,201301.0,397.0,GHL,395259104,0.132910,285.079,396,NaN,NaN,NaN,NaN
28902,90164.0,201401.0,409.0,GHL,395259104,-0.103383,287.152,408,NaN,NaN,NaN,NaN
28903,90164.0,201501.0,421.0,GHL,395259104,-0.154128,275.234,420,NaN,NaN,NaN,NaN
28904,90164.0,201601.0,433.0,GHL,395259104,-0.168822,261.560,432,NaN,NaN,NaN,NaN
28905,90164.0,201701.0,445.0,GHL,395259104,0.066787,335.519,444,NaN,NaN,NaN,NaN
28906,90164.0,201801.0,457.0,GHL,395259104,-0.048718,239.182,456,NaN,NaN,NaN,NaN


In [38]:
final2 = pd.read_csv('merged_final.csv')
final2

,permno,yyyymm,monthid,ticker,gvkey,cusip,naics,gsubind,IM,range_20,...,ret_f3,ret_f4,ret_f5,ret_f6,ret_f7,ret_f8,ret_f9,ret_f10,ret_f11,ret_f12
0,10026,198603.0,75.0,JJSF,12825,466032109,311812,30202030,0.201498,0.146460,...,-0.156250,-0.375000,-0.066667,-0.166667,0.114286,0.051282,-0.048780,0.615385,0.031746,0.030769
1,10026,198604.0,76.0,JJSF,12825,466032109,311812,30202030,0.985464,0.158181,...,-0.375000,-0.066667,-0.166667,0.114286,0.051282,-0.048780,0.615385,0.031746,0.030769,-0.119403
2,10026,198605.0,77.0,JJSF,12825,466032109,311812,30202030,0.715998,0.158346,...,-0.066667,-0.166667,0.114286,0.051282,-0.048780,0.615385,0.031746,0.030769,-0.119403,-0.042373
3,10026,198606.0,78.0,JJSF,12825,466032109,311812,30202030,0.672322,0.126541,...,-0.166667,0.114286,0.051282,-0.048780,0.615385,0.031746,0.030769,-0.119403,-0.042373,0.159292
4,10026,198607.0,79.0,JJSF,12825,466032109,311812,30202030,0.638727,0.127120,...,0.114286,0.051282,-0.048780,0.615385,0.031746,0.030769,-0.119403,-0.042373,0.159292,0.114504
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401583,93429,201802.0,458.0,CBOE,184500,12503M108,523210,40203040,0.516443,0.089356,...,-0.083817,0.066728,-0.066686,0.040976,-0.048016,0.176011,-0.043598,-0.090968,-0.046612,0.031629
401584,93429,201803.0,459.0,CBOE,184500,12503M108,523210,40203040,0.562365,0.340422,...,0.066728,-0.066686,0.040976,-0.048016,0.176011,-0.043598,-0.090968,-0.046612,0.031629,-0.004900
401585,93429,201804.0,460.0,CBOE,184500,12503M108,523210,40203040,0.544807,0.193830,...,-0.066686,0.040976,-0.048016,0.176011,-0.043598,-0.090968,-0.046612,0.031629,-0.004900,0.064648
401586,93429,201805.0,461.0,CBOE,184500,12503M108,523210,40203040,0.455098,0.122062,...,0.040976,-0.048016,0.176011,-0.043598,-0.090968,-0.046612,0.031629,-0.004900,0.064648,0.071253


In [39]:
salesDf['year'] = salesDf['yyyymm'] // 100
final2['year'] = final2['yyyymm'] // 100

In [40]:
salesDf.tail(50)

,permno,yyyymm,monthid,ticker,cusip,RET,sale,salemonthid,salesgrowth,annual decile,Sr,5 year decile,year
28897,90164.0,200901.0,349.0,GHL,395259104,-0.068081,221.873,348,NaN,NaN,0.000000,0.0,2009.0
28898,90164.0,201001.0,361.0,GHL,395259104,-0.030409,298.645,360,NaN,NaN,NaN,NaN,2010.0
28899,90164.0,201101.0,373.0,GHL,395259104,-0.150098,278.329,372,NaN,NaN,NaN,NaN,2011.0
28900,90164.0,201201.0,385.0,GHL,395259104,0.280176,293.993,384,NaN,NaN,NaN,NaN,2012.0
28901,90164.0,201301.0,397.0,GHL,395259104,0.132910,285.079,396,NaN,NaN,NaN,NaN,2013.0
28902,90164.0,201401.0,409.0,GHL,395259104,-0.103383,287.152,408,NaN,NaN,NaN,NaN,2014.0
28903,90164.0,201501.0,421.0,GHL,395259104,-0.154128,275.234,420,NaN,NaN,NaN,NaN,2015.0
28904,90164.0,201601.0,433.0,GHL,395259104,-0.168822,261.560,432,NaN,NaN,NaN,NaN,2016.0
28905,90164.0,201701.0,445.0,GHL,395259104,0.066787,335.519,444,NaN,NaN,NaN,NaN,2017.0
28906,90164.0,201801.0,457.0,GHL,395259104,-0.048718,239.182,456,NaN,NaN,NaN,NaN,2018.0


In [41]:
final2 = pd.merge(final2, salesDf[['permno', 'year', 'salesgrowth', 'Sr']], on=['permno', 'year'], how='inner')
final2

,permno,yyyymm,monthid,ticker,gvkey,cusip,naics,gsubind,IM,range_20,...,ret_f6,ret_f7,ret_f8,ret_f9,ret_f10,ret_f11,ret_f12,year,salesgrowth,Sr
0,10026,198603.0,75.0,JJSF,12825,466032109,311812,30202030,0.201498,0.146460,...,-0.166667,0.114286,0.051282,-0.048780,0.615385,0.031746,0.030769,1986.0,0.0,0.0
1,10026,198604.0,76.0,JJSF,12825,466032109,311812,30202030,0.985464,0.158181,...,0.114286,0.051282,-0.048780,0.615385,0.031746,0.030769,-0.119403,1986.0,0.0,0.0
2,10026,198605.0,77.0,JJSF,12825,466032109,311812,30202030,0.715998,0.158346,...,0.051282,-0.048780,0.615385,0.031746,0.030769,-0.119403,-0.042373,1986.0,0.0,0.0
3,10026,198606.0,78.0,JJSF,12825,466032109,311812,30202030,0.672322,0.126541,...,-0.048780,0.615385,0.031746,0.030769,-0.119403,-0.042373,0.159292,1986.0,0.0,0.0
4,10026,198607.0,79.0,JJSF,12825,466032109,311812,30202030,0.638727,0.127120,...,0.615385,0.031746,0.030769,-0.119403,-0.042373,0.159292,0.114504,1986.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346277,93429,201802.0,458.0,CBOE,184500,12503M108,523210,40203040,0.516443,0.089356,...,0.040976,-0.048016,0.176011,-0.043598,-0.090968,-0.046612,0.031629,2018.0,NaN,NaN
346278,93429,201803.0,459.0,CBOE,184500,12503M108,523210,40203040,0.562365,0.340422,...,-0.048016,0.176011,-0.043598,-0.090968,-0.046612,0.031629,-0.004900,2018.0,NaN,NaN
346279,93429,201804.0,460.0,CBOE,184500,12503M108,523210,40203040,0.544807,0.193830,...,0.176011,-0.043598,-0.090968,-0.046612,0.031629,-0.004900,0.064648,2018.0,NaN,NaN
346280,93429,201805.0,461.0,CBOE,184500,12503M108,523210,40203040,0.455098,0.122062,...,-0.043598,-0.090968,-0.046612,0.031629,-0.004900,0.064648,0.071253,2018.0,NaN,NaN


In [42]:
final2.tail(50)

,permno,yyyymm,monthid,ticker,gvkey,cusip,naics,gsubind,IM,range_20,...,ret_f6,ret_f7,ret_f8,ret_f9,ret_f10,ret_f11,ret_f12,year,salesgrowth,Sr
346232,93429,201405.0,413.0,CBOE,184500,12503M108,523210,40203040,0.503900,0.157725,...,0.020020,0.058588,0.016556,-0.065612,-0.043728,-0.019772,0.043718,2014.0,NaN,0.0
346233,93429,201406.0,414.0,CBOE,184500,12503M108,523210,40203040,0.435023,0.129731,...,0.058588,0.016556,-0.065612,-0.043728,-0.019772,0.043718,-0.022215,2014.0,NaN,0.0
346234,93429,201407.0,415.0,CBOE,184500,12503M108,523210,40203040,0.396529,0.137218,...,0.016556,-0.065612,-0.043728,-0.019772,0.043718,-0.022215,0.083188,2014.0,NaN,0.0
346235,93429,201408.0,416.0,CBOE,184500,12503M108,523210,40203040,0.401233,0.114055,...,-0.065612,-0.043728,-0.019772,0.043718,-0.022215,0.083188,0.020652,2014.0,NaN,0.0
346236,93429,201409.0,417.0,CBOE,184500,12503M108,523210,40203040,0.430280,0.087325,...,-0.043728,-0.019772,0.043718,-0.022215,0.083188,0.020652,0.064022,2014.0,NaN,0.0
346237,93429,201410.0,418.0,CBOE,184500,12503M108,523210,40203040,0.438295,0.077610,...,-0.019772,0.043718,-0.022215,0.083188,0.020652,0.064022,-0.000596,2014.0,NaN,0.0
346238,93429,201411.0,419.0,CBOE,184500,12503M108,523210,40203040,0.399806,0.158321,...,0.043718,-0.022215,0.083188,0.020652,0.064022,-0.000596,0.077118,2014.0,NaN,0.0
346239,93429,201412.0,420.0,CBOE,184500,12503M108,523210,40203040,0.469841,0.135281,...,-0.022215,0.083188,0.020652,0.064022,-0.000596,0.077118,-0.098047,2014.0,NaN,0.0
346240,93429,201501.0,421.0,CBOE,184500,12503M108,523210,40203040,0.468677,0.130436,...,0.083188,0.020652,0.064022,-0.000596,0.077118,-0.098047,0.026502,2015.0,NaN,0.0
346241,93429,201502.0,422.0,CBOE,184500,12503M108,523210,40203040,0.455090,0.150142,...,0.020652,0.064022,-0.000596,0.077118,-0.098047,0.026502,-0.061843,2015.0,NaN,0.0


In [43]:
final2 = final2.rename(columns={'salesgrowth': 'Sg'})

In [44]:
final2 = final2.dropna(subset=['Sr', 'Sg'])
final2

,permno,yyyymm,monthid,ticker,gvkey,cusip,naics,gsubind,IM,range_20,...,ret_f6,ret_f7,ret_f8,ret_f9,ret_f10,ret_f11,ret_f12,year,Sg,Sr
0,10026,198603.0,75.0,JJSF,12825,466032109,311812,30202030,0.201498,0.146460,...,-0.166667,0.114286,0.051282,-0.048780,0.615385,0.031746,0.030769,1986.0,0.000000,0.0
1,10026,198604.0,76.0,JJSF,12825,466032109,311812,30202030,0.985464,0.158181,...,0.114286,0.051282,-0.048780,0.615385,0.031746,0.030769,-0.119403,1986.0,0.000000,0.0
2,10026,198605.0,77.0,JJSF,12825,466032109,311812,30202030,0.715998,0.158346,...,0.051282,-0.048780,0.615385,0.031746,0.030769,-0.119403,-0.042373,1986.0,0.000000,0.0
3,10026,198606.0,78.0,JJSF,12825,466032109,311812,30202030,0.672322,0.126541,...,-0.048780,0.615385,0.031746,0.030769,-0.119403,-0.042373,0.159292,1986.0,0.000000,0.0
4,10026,198607.0,79.0,JJSF,12825,466032109,311812,30202030,0.638727,0.127120,...,0.615385,0.031746,0.030769,-0.119403,-0.042373,0.159292,0.114504,1986.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
346187,93427,201908.0,476.0,FN,179583,G3323L100,3344,45203020,0.515433,0.124486,...,-0.125635,-0.010160,0.150110,0.018964,-0.023772,0.163569,-0.039240,2019.0,0.154826,0.0
346188,93427,201909.0,477.0,FN,179583,G3323L100,3344,45203020,0.427361,0.282884,...,-0.010160,0.150110,0.018964,-0.023772,0.163569,-0.039240,-0.096733,2019.0,0.154826,0.0
346189,93427,201910.0,478.0,FN,179583,G3323L100,3344,45203020,0.458960,0.219210,...,0.150110,0.018964,-0.023772,0.163569,-0.039240,-0.096733,-0.047755,2019.0,0.154826,0.0
346190,93427,201911.0,479.0,FN,179583,G3323L100,3344,45203020,0.452644,0.188740,...,0.018964,-0.023772,0.163569,-0.039240,-0.096733,-0.047755,0.138121,2019.0,0.154826,0.0


In [45]:
final

,permno,yyyymm,monthid,ticker,gvkey,cusip,naics,gsubind,IM,range_20,...,ret_f6,ret_f7,ret_f8,ret_f9,ret_f10,ret_f11,ret_f12,Rs,tes,Nei
0,10026,199001.0,121.0,JJSF,12825,466032109,311812,30202030,0.511680,0.407641,...,0.030303,-0.088235,-0.209677,-0.040816,0.489362,-0.100000,0.015873,-0.224550,0.068135,2
1,10026,199002.0,122.0,JJSF,12825,466032109,311812,30202030,0.524748,0.444449,...,-0.088235,-0.209677,-0.040816,0.489362,-0.100000,0.015873,0.375000,-0.225071,0.068135,2
2,10026,199003.0,123.0,JJSF,12825,466032109,311812,30202030,0.299829,0.520924,...,-0.209677,-0.040816,0.489362,-0.100000,0.015873,0.375000,0.056818,-1.219253,0.068135,0
3,10026,199004.0,124.0,JJSF,12825,466032109,311812,30202030,0.111335,0.303839,...,-0.040816,0.489362,-0.100000,0.015873,0.375000,0.056818,0.193548,-1.161494,-0.125056,0
4,10026,199005.0,125.0,JJSF,12825,466032109,311812,30202030,0.184506,0.378662,...,0.489362,-0.100000,0.015873,0.375000,0.056818,0.193548,-0.045045,-1.148628,-0.125056,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
297389,93429,201801.0,457.0,CBOE,184500,12503M108,523210,40203040,0.568230,0.089902,...,-0.066686,0.040976,-0.048016,0.176011,-0.043598,-0.090968,-0.046612,2.341509,0.407873,1
297390,93429,201803.0,459.0,CBOE,184500,12503M108,523210,40203040,0.562365,0.340422,...,-0.048016,0.176011,-0.043598,-0.090968,-0.046612,0.031629,-0.004900,2.157655,0.407873,1
297391,93429,201804.0,460.0,CBOE,184500,12503M108,523210,40203040,0.544807,0.193830,...,0.176011,-0.043598,-0.090968,-0.046612,0.031629,-0.004900,0.064648,2.130446,-10.138196,1
297392,93429,201805.0,461.0,CBOE,184500,12503M108,523210,40203040,0.455098,0.122062,...,-0.043598,-0.090968,-0.046612,0.031629,-0.004900,0.064648,0.071253,2.112371,-10.138196,1


In [46]:
final.to_csv('factorsA7_A8_A10.csv', index=False)

In [47]:
final2.to_csv('factorsA218_A219.csv', index=False)